In [ ]:
import wandb
from ultralytics import YOLO
import cv2
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
!nvidia-smi

## Login into Weights and Bias

In [ ]:
WANDB_API_KEY = os.getenv("WANDB_API_KEY")
wandb.login(key=WANDB_API_KEY)

##  Model Training

In [ ]:
# Using YOLO's pretrained model architecture and weights for training
model=YOLO('yolov8m-seg.yaml').load('weights/yolov8m-seg.pt')

### Model Training Arguments

In [ ]:
args = dict(
    data="/Users/shaunliew/Documents/endoinsight-ai/config.yaml",  # Using the config_path you already have
    batch=8,
    device="0",
    epochs=30,  # As per your original code
    workers=8,
    optimizer='AdamW',
    save_period=5,
    name='endo_segmentation',
    save=True,
    plots=True,
    patience=5,
    verbose=True,
    iou=0.4,
    conf=0.01,
    degrees=60, # Augmentation arguments
    shear=30,
    perspective=0.0005
)

In [ ]:
#Initiating Model Training
results = model.train(**args)